# Accessing the DE-STRESS Database

## Imported .dump file with all DE-STRESS data into local Postgres server

In [ ]:
pg_restore -U postgres -d local_DE-STRESS_DB file_path/bigstructure.dump

## Extracted wanted information from DE-STRESS into pickle file

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session
import multiprocessing
import time
import pickle
from functools import partial
from db_settings import URL
from DB_library import ChainModel, StateModel, RosettaResultsModel, BiolUnitModel, PdbModel, BudeFFResultsModel,EvoEF2ResultsModel,DFIRE2ResultsModel,Aggrescan3DResultsModel
from Rotatory_library import rotatory


engine=create_engine( URL, convert_unicode=True,pool_pre_ping=True)
sessions=sessionmaker()
session=sessions(bind=engine)

def querying(list_of_datapoints, iteration_index):   # Function for  querying through multiprocessing instead of a for loop
    
    if iteration_index<len(list_of_datapoints):         # Prevents an 'IndexError: list index out of range' because when query slides to the last window it will iterate over 10 000 even though that query.all() list is smaller.
        instance=list_of_datapoints[iteration_index]    # This instance is a list where each element is an object of each table queried - query(ChainModel, StateModel,RosettaResultsModel, BiolUnitModel,PdbModel )
        dict_of_attributes={'Chain ID':instance[0].id, 'Chain State':instance[1].state_number,
        'Mean Pack Density':instance[1].mean_packing_density,'Chain Length':len(instance[0].sequence),
        'Rotatory Bonds':rotatory(instance[0].sequence),'Rosetta Energy': instance[2].total_score, 
        'PDB Code':instance[4].pdb_code, 'Exp Method':instance[4].method, 'Bude Score':instance[5].total_energy,
         'Evo Score':instance[6].total, 'DFire Score':instance[7].total, 'Aggre Score':instance[8].total_value}
        
        return dict_of_attributes

whole_data_list=[]          # List of dictionaries, where each dictionary is an instance with each attribute as a key, and correspondent value
if __name__=='__main__':                 
    query=session.query(ChainModel, StateModel,RosettaResultsModel, BiolUnitModel,
    PdbModel).distinct(ChainModel.sequence).join(ChainModel.state).join(
    StateModel.biol_unit).join(BiolUnitModel.pdb).join(StateModel.rosetta_results)
    
    for number in range((query.count()//10000)+1):  # query object allows to get the length of the query. The .count() method does not work if query.all()
        
        with Session(engine) as session:            # Use with python context manager to separate the transactions between the queries of each iteration of the for loop above
            data=session.query(ChainModel, StateModel,RosettaResultsModel, 
                BiolUnitModel,PdbModel,BudeFFResultsModel,EvoEF2ResultsModel,
                DFIRE2ResultsModel,Aggrescan3DResultsModel).distinct(ChainModel.sequence).join(
                ChainModel.state).join(StateModel.biol_unit).join(BiolUnitModel.pdb).join(
                StateModel.rosetta_results).join(StateModel.budeff_results).join(
                StateModel.evoef2_results).join(StateModel.dfire2_results).join(
                StateModel.aggrescan3d_results).offset(number*10000).limit(10000).all() # Distinct filters the chains so that they are all uniq
            
            with multiprocessing.Pool(7) as pool:
                sliced_data=pool.map(partial(querying, data), range(10000))  # map takes one function and one list, and partial allows to add the data object as a constant.
                whole_data_list+=sliced_data                                 # pool joins the outcome of each iteration within the sliced data into a list (sliced_data). This creates a list of dictionaries of the size of the slice section

    with open('DE-STRESS_data.pickle','wb') as pickle_out: 
        pickle.dump(whole_data_list, pickle_out)                            # Extracted data saved in the 'DE-STRESS_data' pickle file
